In [17]:
import nfl_data_py as nfl
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnnotationBbox
from functools import reduce
import os

In [3]:
# #Just to get a reference of everything
# df = pd.DataFrame(pbp_2022)
# df.to_csv('pbp_2022.csv')

# Loading last N Years

In [4]:
pbp = nfl.import_pbp_data([2019, 2020, 2021, 2022])
# pbp_2021 = nfl.import_pbp_data([2021])
# pbp_2020 = nfl.import_pbp_data([2020])
# pbp_2019 = nfl.import_pbp_data([2019])

2019 done.
2020 done.
2021 done.
2022 done.
Downcasting floats.


# Filtering to scoring plays only

In [5]:
regular_season_last_4 = pbp[(pbp['season_type'] == 'REG')]
regular_season_last_4['year'] = pbp['game_id'].str[:4]
regular_season_last_4 = regular_season_last_4.dropna(subset=['epa', 'posteam', 'defteam'])
#Might need to use this line below - not sure if it will affect anything but need to 
#validate if all the numbers are coming out correct first
#regular_season_2022 = pbp_2022.dropna(subset=['epa', 'posteam', 'defteam'])

#Is this the best way to wrap long lines? With the '\'
#Do I need the parenthesis in every filter criteria?
scoring_last_4 = regular_season_last_4[(regular_season_last_4['rush_touchdown'] == 1) | \
                        (regular_season_last_4['pass_touchdown'] == 1) | \
                        (regular_season_last_4['extra_point_result'] == 'good') | \
                        (regular_season_last_4['field_goal_result'] == 'made')]

#May need to put this back
#Looks like it does not need to be there, leaving it just in case
#scoring_last_4['year'] = scoring_last_4['game_id'].str[:4]
#scoring_last_4.shape


# Rushing Touchdowns
#### Goal is to be able to see a trend over time and get an average
#### Using this to make projections about the upcoming season

In [6]:
#Doing this single year by single year because I do not know how to code
rushingTDs_2022 = scoring_last_4[(scoring_last_4['rush_touchdown'] == 1) & \
            (scoring_last_4['year'] == '2022')] \
            .groupby(['posteam']).sum(numeric_only=True)['rush_touchdown'] \
            .rename('rushTDs2022')
rushingTDs_2021 = scoring_last_4[(scoring_last_4['rush_touchdown'] == 1) & \
            (scoring_last_4['year'] == '2021')] \
            .groupby(['posteam']).sum(numeric_only=True)['rush_touchdown'] \
            .rename('rushTDs2021')
rushingTDs_2020 = scoring_last_4[(scoring_last_4['rush_touchdown'] == 1) & \
            (scoring_last_4['year'] == '2020')] \
            .groupby(['posteam']).sum(numeric_only=True)['rush_touchdown'] \
            .rename('rushTDs2020')
rushingTDs_2019 = scoring_last_4[(scoring_last_4['rush_touchdown'] == 1) & \
            (scoring_last_4['year'] == '2019')] \
            .groupby(['posteam']).sum(numeric_only=True)['rush_touchdown'] \
            .rename('rushTDs2019')

#Merging into a single dataframe
last4RushingTDs = pd.merge(pd.merge(pd.merge(rushingTDs_2022, rushingTDs_2021, on='posteam'),\
        rushingTDs_2020, on='posteam'), rushingTDs_2019, on='posteam')

#Getting the average of the last 4 years
last4RushingTDs.assign(rushTD4Yavg=lambda \
                       x: ((x.rushTDs2022+x.rushTDs2021+x.rushTDs2020+x.rushTDs2019)/4))
pass

# Passing Touchdowns
#### Same as Rushing TD process

In [7]:
#Just doing it one year at a time again

passingTDs_2022 = scoring_last_4[(scoring_last_4['pass_touchdown'] == 1) & \
            (scoring_last_4['year'] == '2022')] \
            .groupby(['posteam']).sum(numeric_only=True)['pass_touchdown'] \
            .rename('passTDs2022')
passingTDs_2021 = scoring_last_4[(scoring_last_4['pass_touchdown'] == 1) & \
            (scoring_last_4['year'] == '2021')] \
            .groupby(['posteam']).sum(numeric_only=True)['pass_touchdown'] \
            .rename('passTDs2021')
passingTDs_2020 = scoring_last_4[(scoring_last_4['pass_touchdown'] == 1) & \
            (scoring_last_4['year'] == '2020')] \
            .groupby(['posteam']).sum(numeric_only=True)['pass_touchdown'] \
            .rename('passTDs2020')
passingTDs_2019 = scoring_last_4[(scoring_last_4['pass_touchdown'] == 1) & \
            (scoring_last_4['year'] == '2019')] \
            .groupby(['posteam']).sum(numeric_only=True)['pass_touchdown'] \
            .rename('passTDs2019')

#Merging into a single dataframe
#Gotta be a better way to indent this stupid thing
last4PassingTDs = pd.merge(pd.merge(pd.merge(passingTDs_2022, passingTDs_2021, on='posteam'),\
        passingTDs_2020, on='posteam'), passingTDs_2019, on='posteam')

#Is this the best way to indent?
last4PassingTDs.assign(passTD4Yavg=lambda
                    x:((x.passTDs2022+x.passTDs2021+x.passTDs2020+x.passTDs2019)/4)) \
                    .head()


,passTDs2022,passTDs2021,passTDs2020,passTDs2019,passTD4Yavg
posteam,,,,,
ARI,17.0,27.0,27.0,20.0,22.75
ATL,17.0,20.0,27.0,29.0,23.25
BAL,19.0,21.0,27.0,37.0,26.00
BUF,35.0,36.0,40.0,21.0,33.00
CAR,16.0,14.0,16.0,17.0,15.75


# Passing Yards

In [8]:
regular_season_last_4 = pbp[(pbp['season_type'] == 'REG') & \
                        ((pbp['play_type_nfl'] == 'PASS') | \
                        (pbp['play_type_nfl'] == 'SACK'))]
regular_season_last_4['year'] = pbp['game_id'].str[:4]
                
#passYards_2022 = regular_season_last_4[(regular_season_last_4['sack'] == 0)]
passYards_2022 = regular_season_last_4[(regular_season_last_4['year'] == '2022')] \
            .groupby(['posteam']).sum(numeric_only=True)['yards_gained'] \
            .rename('passYards2022')
passYards_2021 = regular_season_last_4[(regular_season_last_4['year'] == '2021')] \
            .groupby(['posteam']).sum(numeric_only=True)['yards_gained'] \
            .rename('passYards2021')
passYards_2020 = regular_season_last_4[(regular_season_last_4['year'] == '2020')] \
            .groupby(['posteam']).sum(numeric_only=True)['yards_gained'] \
            .rename('passYards2020')
passYards_2019 = regular_season_last_4[(regular_season_last_4['year'] == '2019')] \
            .groupby(['posteam']).sum(numeric_only=True)['yards_gained'] \
            .rename('passYards2019')

last4PassingYards = pd.merge(pd.merge(pd.merge(passYards_2022, passYards_2021, on='posteam'),\
        passYards_2020, on='posteam'), passYards_2019, on='posteam')

last4PassingYards.head(3)

,passYards2022,passYards2021,passYards2020,passYards2019
posteam,,,,
ARI,3626.0,4276.0,3916.0,3477.0
ATL,2699.0,3713.0,4363.0,4714.0
BAL,3040.0,3961.0,2739.0,3225.0


# Rushing Yards

In [9]:
rushing_last_4 = pbp[(pbp['season_type'] == 'REG')]
rushing_last_4['year'] = pbp['game_id'].str[:4]
rushing_last_4 = rushing_last_4.dropna(subset=['epa', 'posteam', 'defteam'])
rushing_last_4 = rushing_last_4[(rushing_last_4['rush_attempt'] == 1)]
                

rushYards_2022 = rushing_last_4[(rushing_last_4['year'] == '2022')] \
            .groupby(['posteam']).sum(numeric_only=True)['rushing_yards'] \
            .rename('rushYards2022')
rushYards_2021 = rushing_last_4[(rushing_last_4['year'] == '2021')] \
            .groupby(['posteam']).sum(numeric_only=True)['rushing_yards'] \
            .rename('rushYards2021')
rushYards_2020 = rushing_last_4[(rushing_last_4['year'] == '2020')] \
            .groupby(['posteam']).sum(numeric_only=True)['rushing_yards'] \
            .rename('rushYards2020')
rushYards_2019 = rushing_last_4[(rushing_last_4['year'] == '2019')] \
            .groupby(['posteam']).sum(numeric_only=True)['rushing_yards'] \
            .rename('rushYards2019')

last4RushYards = pd.merge(pd.merge(pd.merge(rushYards_2022, rushYards_2021, on='posteam'),\
        rushYards_2020, on='posteam'), rushYards_2019, on='posteam')
#Getting the average
last4RushYards.assign(RushYards4Yavg=lambda \
                      x: ((x.rushYards2022+x.rushYards2021+x.rushYards2020+x.rushYards2019)/4))


,rushYards2022,rushYards2021,rushYards2020,rushYards2019,RushYards4Yavg
posteam,,,,,
ARI,1873.0,2076.0,2237.0,1990.0,2044.00
ATL,2718.0,1451.0,1532.0,1361.0,1765.50
BAL,2720.0,2479.0,3070.0,3287.0,2889.00
BUF,2232.0,2205.0,1723.0,2054.0,2053.50
CAR,2210.0,1842.0,1704.0,1819.0,1893.75
CHI,3014.0,2015.0,1647.0,1458.0,2033.50
CIN,1528.0,1742.0,1668.0,1517.0,1613.75
CLE,2490.0,2471.0,2372.0,1901.0,2308.50
DAL,2298.0,2104.0,1788.0,2153.0,2085.75


 # Play Count

In [16]:
play_count_last_4 = pbp[pbp['season_type'] == 'REG']
play_count_last_4['year'] = pbp['game_id'].str[:4]

#2022 Plays ------------------------------------------------------------------------
run_count_2022 = play_count_last_4[(play_count_last_4['play_type_nfl'] == 'RUSH') & \
                                (play_count_last_4['year'] == '2022')] \
                                .groupby(['posteam']).count()['year'].rename('run_plays_2022')
 
pass_count_2022 = play_count_last_4[((play_count_last_4['play_type_nfl'] == 'PASS') | \
                                (play_count_last_4['play_type_nfl'] == 'SACK')) & \
                                (play_count_last_4['year'] == '2022')] \
                                .groupby(['posteam']).count()['year'].rename('pass_plays_2022')

play_count_2022 = pd.merge(run_count_2022, pass_count_2022, on='posteam')
play_count_2022.assign(total_plays_2022=lambda x: x.run_plays_2022 + x.pass_plays_2022
                    ,run_percent_2022=lambda x: x.run_plays_2022 / x.total_plays_2022
                    ,pass_percent_2022=lambda x: x.pass_plays_2022 / x.total_plays_2022)

#2021 Plays ------------------------------------------------------------------------
run_count_2021 = play_count_last_4[(play_count_last_4['play_type_nfl'] == 'RUSH') & \
                                (play_count_last_4['year'] == '2021')] \
                                .groupby(['posteam']).count()['year'].rename('run_plays_2021')
 
pass_count_2021 = play_count_last_4[((play_count_last_4['play_type_nfl'] == 'PASS') | \
                                (play_count_last_4['play_type_nfl'] == 'SACK')) & \
                                (play_count_last_4['year'] == '2021')] \
                                .groupby(['posteam']).count()['year'].rename('pass_plays_2021')

play_count_2021 = pd.merge(run_count_2021, pass_count_2021, on='posteam')
play_count_2021.assign(total_plays_2021=lambda x: x.run_plays_2021 + x.pass_plays_2021
                    ,run_percent_2021=lambda x: x.run_plays_2021 / x.total_plays_2021
                    ,pass_percent_2021=lambda x: x.pass_plays_2021 / x.total_plays_2021)

#2020 Plays ------------------------------------------------------------------------
run_count_2020 = play_count_last_4[(play_count_last_4['play_type_nfl'] == 'RUSH') & \
                                (play_count_last_4['year'] == '2020')] \
                                .groupby(['posteam']).count()['year'].rename('run_plays_2020')
 
pass_count_2020 = play_count_last_4[((play_count_last_4['play_type_nfl'] == 'PASS') | \
                                (play_count_last_4['play_type_nfl'] == 'SACK')) & \
                                (play_count_last_4['year'] == '2020')] \
                                .groupby(['posteam']).count()['year'].rename('pass_plays_2020')

play_count_2020 = pd.merge(run_count_2020, pass_count_2020, on='posteam')
play_count_2020.assign(total_plays_2020=lambda x: x.run_plays_2020 + x.pass_plays_2020
                    ,run_percent_2020=lambda x: x.run_plays_2020 / x.total_plays_2020
                    ,pass_percent_2020=lambda x: x.pass_plays_2020 / x.total_plays_2020)

#2019 Plays ------------------------------------------------------------------------
run_count_2019 = play_count_last_4[(play_count_last_4['play_type_nfl'] == 'RUSH') & \
                                (play_count_last_4['year'] == '2019')] \
                                .groupby(['posteam']).count()['year'].rename('run_plays_2019')
 
pass_count_2019 = play_count_last_4[((play_count_last_4['play_type_nfl'] == 'PASS') | \
                                (play_count_last_4['play_type_nfl'] == 'SACK')) & \
                                (play_count_last_4['year'] == '2019')] \
                                .groupby(['posteam']).count()['year'].rename('pass_plays_2019')

play_count_2019 = pd.merge(run_count_2019, pass_count_2019, on='posteam')
play_count_2019.assign(total_plays_2019=lambda x: x.run_plays_2019 + x.pass_plays_2019
                    ,run_percent_2019=lambda x: x.run_plays_2019 / x.total_plays_2019
                    ,pass_percent_2019=lambda x: x.pass_plays_2019 / x.total_plays_2019)

#Need to merge them all together
last4PlayCount = pd.merge(pd.merge(pd.merge(play_count_2022, play_count_2021, on='posteam'),\
        play_count_2020, on='posteam'), play_count_2019, on='posteam')

last4PlayCount.head()

,run_plays_2022,pass_plays_2022,run_plays_2021,pass_plays_2021,run_plays_2020,pass_plays_2020,run_plays_2019,pass_plays_2019
posteam,,,,,,,,
ARI,434,710,496,630,479,604,396,604
ATL,559,452,393,613,409,669,362,734
BAL,526,526,517,668,555,438,596,468
BUF,430,607,461,682,411,623,465,553
CAR,483,493,455,651,407,586,386,691


# Final report?

In [31]:
#Combining all of the dataframes from above
HistoricalDFs = [last4RushingTDs, last4PassingTDs, last4PassingYards,\
                     last4RushYards, last4PlayCount]

HistoricalScoring = reduce(lambda left,right: pd.merge(left,right,on=['posteam'],\
                                                       how='outer'), HistoricalDFs)

HistoricalScoring

,rushTDs2022,rushTDs2021,rushTDs2020,rushTDs2019,passTDs2022,passTDs2021,passTDs2020,passTDs2019,passYards2022,passYards2021,...,rushYards2020,rushYards2019,run_plays_2022,pass_plays_2022,run_plays_2021,pass_plays_2021,run_plays_2020,pass_plays_2020,run_plays_2019,pass_plays_2019
posteam,,,,,,,,,,,,,,,,,,,,,
ARI,15.0,23.0,22.0,18.0,17.0,27.0,27.0,20.0,3626.0,4276.0,...,2237.0,1990.0,434,710,496,630,479,604,396,604
ATL,17.0,11.0,13.0,10.0,17.0,20.0,27.0,29.0,2699.0,3713.0,...,1532.0,1361.0,559,452,393,613,409,669,362,734
BAL,14.0,18.0,24.0,21.0,19.0,21.0,27.0,37.0,3040.0,3961.0,...,3070.0,3287.0,526,526,517,668,555,438,596,468
BUF,15.0,20.0,16.0,13.0,35.0,36.0,40.0,21.0,4129.0,4284.0,...,1723.0,2054.0,430,607,461,682,411,623,465,553
CAR,16.0,17.0,19.0,20.0,16.0,14.0,16.0,17.0,2996.0,3239.0,...,1704.0,1819.0,483,493,455,651,407,586,386,691
CHI,18.0,14.0,12.0,8.0,19.0,16.0,26.0,20.0,2219.0,3207.0,...,1647.0,1458.0,558,435,475,600,393,650,395,625
CIN,14.0,16.0,13.0,9.0,35.0,36.0,19.0,18.0,4240.0,4403.0,...,1668.0,1517.0,399,654,436,610,411,629,385,664
CLE,19.0,20.0,21.0,15.0,19.0,21.0,27.0,22.0,3444.0,3320.0,...,2372.0,1901.0,532,584,485,569,495,527,393,580
DAL,24.0,15.0,14.0,18.0,28.0,40.0,25.0,30.0,3736.0,4800.0,...,1788.0,2153.0,531,583,473,680,430,683,449,620


In [32]:
HistoricalScoring.to_csv('HistoricalScoring.csv')
#last4PassingTDs.to_csv('Last_4Y_passTD.csv')
#last4PassingTDs.to_csv('Last_4Y_passTD.csv')
#last4PassingTDs.to_csv('Last_4Y_passTD.csv')